## Assignment 3 - Preprocessing data


### Goals

- First: the original hashtags have been blanked out from the tweet. Build a predictive model that can predict the hashtag used based on the rest of the tweet text
- Second: show that your model can make predictions in a deployed streaming setting using Spark


#### Tasks
Using Spark, your task for this assignment is as follows:

- [X] Construct a historical data set using the provided stream
- [ ] Construct a predictive model to predict the used hashtag based on the tweet text
     - [X] load historical data 
     - [X] set histdata (.json) as dataframe
     - [ ]  
- [ ] Use your trained model to show you can make predictions as the stream comes in

The stream is text-based with each line containing one tweet (one instance) formatted as a JSON dictionary with the following keys (features): "tweet_id" (id of the tweet, not to be used), "tweet_text" (the tweet with the hashtags blanked out) and "label" (the target) You can use extra data and libraries if you want, but this is not required You are strongly encouraged to build your model using spark.ml (MLlib), but you can use scikit-learn as a fallback if things don't work out
I.e. show that you can connect to the data source, preprocess/featurize incoming tweets, have your model predict the label, and show it, similar to "spark_streaming_example_predicting.py.ipynb" (but hopefully using a smarter, real predictive model) This means that you'll need to look for a way to save and load your trained model

In [ ]:
!pip install spacy

In [ ]:
!pip install wordcloud

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import glob

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark = SparkSession.builder \
    .getOrCreate()

In [6]:
#import glob
#print(len(glob.glob("C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics/myoutput_draft/-*")))

#### Load data

What a nightmare!!

sc.textFile along with spark.read.json collapse with the amount of tweets collected.

In [8]:
#data = sc.textFile("C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics/myoutput_draft/-*",minPartitions=8 )
#data.first() 

In [9]:
#df = spark.read.json(data)

First we have to get all the files that were collected. Then, we filter only for the ones that have tweet info (files that do not include _SUCCESS or crc).

In [10]:
parent_path = "C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics"
files=[]

for (dirpath, dirnames, filenames) in os.walk(parent_path):
    files.extend(os.path.join(dirpath, filename) for filename in filenames)

files_filter=[]

for ad in files: 
    if not "_SUCCESS" in ad and not "crc" in ad and not ".csv" in ad:
        files_filter.append(ad)

In [ ]:
print(len(files_filter))

Update: Colapsed at 274. why? loop was using 
colapsed by me at 775

In [ ]:
df1 = spark.read.json(files_filter[0])

i = 1
while i < len(files_filter):
  # create the csv writer object
    for dirpath in files_filter[i:i+1]:
        file=spark.read.json(dirpath)
        try:
            df3 = file.toPandas()
        #    df1=df1.union(file)         
        except TypeError:
            pass   
    df3.to_csv("C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics/tweet_05.csv", mode= 'a', index= False, header = False)
    i += 1
    print(i, "has been read")
else:
    print("I'm done!! (in theory)")

In [ ]:
gc.collect()

['C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_00.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_01.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_02.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_03.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_04.csv',
 'C:/Users/Marce/OneDrive - KU Leuven/Advanced Analytics\\tweet_05.csv']

,label,tweet_id,tweet_text
11051,#inflation,1385683130338643968,Happy Friday! Today Chat chats inflation - we'...
3906,#vaccine,1382328808066281473,EMA is working closely with the @US_FDA and ot...
2129,#china,1382010563392282624,@DrTedros and #███████ just destroyed our live...
1218,#biden,1381664559711277062,@AntifascistSec @7News Lol. Do your parents kn...
6967,#china,1382738195847155713,'You Have The AUDACITY To Talk About #███████?...


(11577, 3)